In [1]:
from __future__ import annotations

import json
import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RelMap,
    SubMap,
    SelIndex,
    DB
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
async def dummy_loader(name: str) -> dict:
    await sleep(0.2)
    return {
        'name': name,
        'age': 20
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RelMap(
            rel=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RelMap(
                    rel=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RelMap(
                            rel=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RelMap(
                    rel=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    link=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [8]:
with cProfile.Profile() as pr:
  db = DB()
  rec = await data_source.load([data], db=db)
  
  stats = pstats.Stats(pr)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\lworm\AppData\Local\Temp\ipykernel_18888\2867539664.py:3 in <module>                    │
│                                                                                                  │
│   1 with cProfile.Profile() as pr:                                                               │
│   2   db = DB()                                                                                  │
│ ❱ 3   rec = await data_source.load([data], db=db)                                                │
│   4                                                                                              │
│   5   stats = pstats.Stats(pr)                                                                   │
│   6                                                                                              │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\data_source.py:756 in load                   │
│                                                                                                  │
│   753 │   │   db = db if db is not None else DB()                                                │
│   754 │   │   in_data: dict[DirectPath, TreeNode] = {(): dat for dat in data}                    │
│   755 │   │   rest_data: RestTreeData = {}                                                       │
│ ❱ 756 │   │   loaded = await _load_records(db, self, in_data, rest_data)                         │
│   757 │   │                                                                                      │
│   758 │   │   # Perform second pass to load remaining data                                       │
│   759 │   │   for rec_map, tree_data in rest_data.items():                                       │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\data_source.py:720 in _load_records          │
│                                                                                                  │
│   717 │   │   │   │   rel_set = cast(                                                            │
│   718 │   │   │   │   │   RecSet[Record[Any], RW, Static, None, Full], rec_set[rel]              │
│   719 │   │   │   │   )                                                                          │
│ ❱ 720 │   │   │   │   rel_set |= {(parent_idx, idx): idx for idx in new_rel_dict.keys()}         │
│   721 │                                                                                          │
│   722 │   return records                                                                         │
│   723                                                                                            │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\base.py:2648 in __ior__                      │
│                                                                                                  │
│   2645 │   │   other: RecSet[RecT, Any, Any, Any, Any] | RecInput[RecT, KeyT2, KeyT2],           │
│   2646 │   ) -> RecSet[RecT, WriteT, BackT, SelT, Full, RecT]:                                   │
│   2647 │   │   """Upserting assignment."""                                                       │
│ ❱ 2648 │   │   self._mutate(other, mode="upsert")                                                │
│   2649 │   │   return cast(RecSet[RecT, WriteT, BackT, SelT, Full, RecT], self)                  │
│   2650 │                                                                                         │
│   2651 │   @overload                                                                             │
│                                                            

In [8]:
[s for s in db[Search]]

[{'term': None, 'result_count': 3}]

In [9]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         100327 function calls (95753 primitive calls) in 0.345 seconds

   Ordered by: cumulative time
   List reduced from 2037 to 94 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.028    0.028 C:\Users\lworm\Repos\py-research\src\py_research\db\base.py:4351(engine)
        2    0.000    0.000    0.004    0.002 C:\Users\lworm\Repos\py-research\src\py_research\db\data_source.py:234(full_map)
        2    0.000    0.000    0.004    0.002 C:\Users\lworm\Repos\py-research\src\py_research\db\data_source.py:350(_push_to_pull_map)
   430/36    0.000    0.000    0.003    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:54(inner)
   430/36    0.001    0.000    0.003    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:69(gen_int_hash)
       68    0.000    0.000    0.003    0.000 C:\Users\lworm\Repos\py-research\src\py_research\reflect\types.py:61(has_type)
        6 